In [1]:
# pip install numpy
import numpy as np
# pip install tqdm
from tqdm import tqdm

In [2]:
D = 10
cf_num = 10
search_range = [-100, 100]

## Objective functions

In [3]:
shift_data = np.loadtxt(fname = 'shift_data.txt')
M_D10 = np.loadtxt(fname = 'M_D10.txt').reshape((cf_num, D, D))
M1 = M_D10[0].reshape((-1))
o = shift_data.reshape((-1))[:cf_num*D]

In [4]:
# Shift Function
def shift(x, o):
    
    xshift = x - o[:x.shape[0]]
        
    return xshift

In [5]:
# Rotate Function
def rotate(x, M1):
    
    xrot = np.zeros((x.shape[0]))
    
    for i in range(x.shape[0]):
        for j in range(x.shape[0]):
            xrot[i] = xrot[i] + x[j] * M1[i*D+j]
    
    return xrot

In [6]:
# Unimodal Function
def sphere(x, o, bias = -1400):
    
    """ Sphere function (CEC2013). """
       
    # Shift
    xshift = shift(x, o)
    
    # Sphere
    f = sum(xshift**2) + bias
    
    return f

In [7]:
# Minimum global
sphere(x = o[:D], o = o[:D])

-1400.0

In [8]:
# Basic Multimodal Function
def rosenbrock(x, o, M1, a = 1, b = 100, bias = -900):
    
    """ Rotated Rosenbrock's function (CEC2013)."""
    
    # Shift
    xshift = shift(x, o)
    # Shrink to the orginal search range
    xshift = 2.048/100*xshift
    # Rotate
    xrot = rotate(xshift, M1)
    # Shift to origin
    xrot = xrot + 1
    
    # Rosenbrock
    f = sum(b*(xrot[1:]-xrot[:-1]**2.0)**2.0 + (a-xrot[:-1])**2.0) + bias
    
    return f

In [9]:
# Minimum global
rosenbrock(x = o[:D], o = o[:D], M1 = M1)

-900.0

## Particle Swarm Optimization (PSO)

In [10]:
class ParticleSwarmOptimization():
    
    """ Global/Local Best Particle Swarm Optimization. """
    
    def __init__(self, n, m, fobj, max_eval, search_range, c1, c2, inertia, constriction, delta, neighborhood):
        
        """
            n: Number of dimensions
            m: Number of particles
            fobj: Unimodal ('sphere') or Multimodal ('rosenbrock')
            max_eval: Maximum number of function evaluations
            search_range: Search space range
            c1: Cognitive factor
            c2: Social factor
            inertia: Inertia weight
            constriction: Constriction coefficient
            delta: Factor for maximum velocity clamping
            neighborhood: Type of neighborhood for exploring solutions (global or local)
        """

        self.n = n
        self.m = m
        self.fobj = fobj
        self.n_eval = 0
        self.max_eval = max_eval
        self.search_range = search_range
        self.c1 = c1
        self.c2 = c2
        self.inertia = inertia
        self.constriction = constriction
        self.delta = delta
        self.neighborhood = neighborhood
        
        # Personal best: best position the particle has visited since the first time step 
        self.pbest = np.zeros((self.m, self.n))
        
        if self.neighborhood == 'global':
            # Global best position
            self.gbest = np.zeros((self.n,))
        else:
            # Best position found in the neighborhood of each particle
            self.gbest = np.zeros((self.m, self.n))
            
        # Minimization problem
        self.best_fitness = float('Inf')
        
    def objective_function(self, x, o = o, M1 = None):
        
        if self.fobj == 'sphere':
            f = sphere(x = x, o = o)
        else:
            f = rosenbrock(x = x, o = o, M1 = M1)
        
        return f
        
    def random_initialization(self, o = o, M1 = None):

        """ Random initialization method. """

        swarm = np.zeros((self.m, self.n))
        velocities = np.zeros((self.m, self.n))
        fitness = np.zeros((self.m,))

        for i in range(self.m):
            
            swarm[i] = np.random.uniform(low = self.search_range[0], high = self.search_range[1], size = self.n)
            # The personal best position of each particle is equal to its initial position
            self.pbest[i] = swarm[i].copy()
            # Evaluating particles
            fitness[i] = self.objective_function(x = swarm[i], o = o, M1 = M1)
            
        return swarm, fitness, velocities
    
    def solve(self, o, M1 = None):
    
        """ Minimizes optimization function. """
        
        # Initializes population with random candidate solutions
        swarm, fitness, velocities = self.random_initialization(o = o, M1 = M1)
        pbest_fitness = fitness.copy()
        
        # Maximum velocity
        v_max = self.delta * (search_range[1] - search_range[0])
        
        # Progress bar
        pbar = tqdm(total = self.max_eval, desc = 'Function evaluations')

        # Until the maximum number of function evaluations is reached
        while self.n_eval < self.max_eval:
        
            # Set the personal best position
            for i in range(self.m):                
                if fitness[i] < pbest_fitness[i]:
                    self.pbest[i] = swarm[i].copy()
                    pbest_fitness[i] = fitness[i]
            
            # Set the global best position (global best)
            if self.neighborhood == 'global':                
                self.gbest = self.pbest[np.argmin(pbest_fitness)]
                
            # Set the global best position (local best)
            else:                
                for i in range(self.m):                    
                    # Ring neighborhood structure
                    neighbors = [((i-1)%self.m), i, ((i+1)%self.m)]
                    neighbor_fitness = pbest_fitness[neighbors]
                    self.gbest[i] = self.pbest[neighbors[np.argmin(neighbor_fitness)]]
                    
            for i in range(self.m):
                
                r1 = np.random.uniform(low = 0, high = 1, size = self.n)
                r2 = np.random.uniform(low = 0, high = 1, size = self.n)
                
                # Update velocities
                if self.neighborhood == 'global':
                    velocities[i] = velocities[i] + (self.c1*r1) * (self.pbest[i] - swarm[i]) + (self.c2*r2) * (self.gbest - swarm[i])
                else:
                    velocities[i] = velocities[i] + (self.c1*r1) * (self.pbest[i] - swarm[i]) + (self.c2*r2) * (self.gbest[i] - swarm[i])
                
                # Update positions
                swarm[i] = swarm[i] + velocities[i]
                
                # Update fitness
                fitness[i] = self.objective_function(x = swarm[i], o = o, M1 = M1)
                
                # Update best solution
                if fitness[i] < self.best_fitness:
                    self.best_fitness = fitness[i]
            
            # Limits for the positions and velocities of a particle
            velocities[velocities > v_max] = v_max
            swarm[swarm < self.search_range[0]] = self.search_range[0]
            swarm[swarm > self.search_range[1]] = self.search_range[1]
                
            # Increases number of function evaluations
            self.n_eval = self.n_eval + self.m
            pbar.update(self.m)

In [11]:
PSO = ParticleSwarmOptimization(n = D, 
                                m = 100, 
                                fobj = 'sphere', 
                                max_eval = 100000, 
                                search_range = search_range,
                                c1 = 2.05,
                                c2 = 2.05,
                                inertia = 0.7,
                                constriction = 0.73,
                                delta = 0.025,
                                neighborhood = 'local')

In [12]:
PSO.solve(o = o, M1 = M1)

Function evaluations: 100%|██████████| 100000/100000 [00:03<00:00, 32979.53it/s]


In [13]:
print('Best fitness:', PSO.best_fitness)

Best fitness: -1371.0268242325376
